<a href="https://colab.research.google.com/github/TeeganKrieger/ChickenScratch/blob/Portenta-Application/Copy_of_Grid_Recognition_Model_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grid Recognition Model Training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install micromlgen

  Created wheel for micromlgen: filename=micromlgen-1.1.28-py3-none-any.whl size=32172 sha256=6f400d2db573836c9d0a0ee569f242377396cf7b895c1d340fbaa0f54fe5d5e1
  Stored in directory: /root/.cache/pip/wheels/6f/98/43/0860b692737af546b9262a8b60ec3690c8bdc51c25e545ae09
Successfully built micromlgen


### Import all needed modules

In [ ]:
import numpy as np
import random
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import json
from types import SimpleNamespace
import seaborn; seaborn.set()
from keras.callbacks import EarlyStopping
import tensorflow as tf

MODELS_DIR = 'models/'
MODEL_TF = MODELS_DIR + 'model'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'
# TensorFlow is an open source machine learning library

from micromlgen import port

## Building The Model

In [ ]:
model = Sequential()
model.add(Conv2D(8, (3, 2), activation='relu', input_shape=(240, 320, 3)))
model.add(Conv2D(16, (3, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(16, (3, 2), activation='relu'))
model.add(Conv2D(8, (3, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(9, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 238, 319, 8)       152       
                                                                 
 conv2d_1 (Conv2D)           (None, 236, 318, 16)      784       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 118, 159, 16)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 116, 158, 16)      1552      
                                                                 
 conv2d_3 (Conv2D)           (None, 114, 157, 8)       776       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 57, 78, 8)        0         
 2D)                                                    

## Image and Config Loading

In [ ]:
#Loads an image and its cfg file
def importImage(path, X, Y):
    #Load the image
    img = load_img(path)
    arr = img_to_array(img)
    
    #Load the cfg
    configFile = open(path[:-4] + '.cfg')
    configRaw = configFile.read()
    config = json.loads(configRaw, object_hook=lambda d: SimpleNamespace(**d)) 
    
    cfgArr = [config.valid, config.x1, config.y1, config.x2, config.y2, config.x3, config.y3, config.x4, config.y4]
    X.append(arr)
    Y.append(cfgArr)
    return X, Y

In [ ]:
def loadAllTrainingData(data_path):
    X = []
    Y = []
    for root, dirs, files in os.walk(data_path, topdown=False):
        for f in files:
            if (f.endswith(".jpg")):
                X, Y = importImage(os.path.join(root, f), X, Y)
    return X, Y

## Training Data Splitting

In [ ]:
def splitTrainingValidation(X, Y, split_size=0.2):
    if len(X) == len(Y):
        splitCount = int(len(X) * split_size)
        
        used_indices = []
        
        X_set = []
        X_val = []
        Y_set = []
        Y_val = []
        
        while len(X_val) < splitCount:
            index = random.randint(0, len(X)-1)
            if index not in used_indices:
                X_val.append(X[index])
                Y_val.append(Y[index])
                used_indices.append(index)
        
        for i in range(0, len(X)-1):
            if i not in used_indices:
                X_set.append(X[i])
                Y_set.append(Y[i])
                
        return np.array(X_set), np.array(Y_set), np.array(X_val), np.array(Y_val)

## Loading Training Data and Training

In [ ]:
X, Y = loadAllTrainingData("/content/drive/MyDrive/Traing Data Preprocessing/Training Data/Grid")
X, Y, X_val, Y_val = splitTrainingValidation(X, Y, split_size=0.2)

print(X)
print(Y)
print(X_val.shape)
print(Y_val.shape)

[[[[32. 32. 32.]
   [36. 36. 36.]
   [32. 32. 32.]
   ...
   [ 6.  6.  6.]
   [ 6.  6.  6.]
   [ 6.  6.  6.]]

  [[35. 35. 35.]
   [39. 39. 39.]
   [34. 34. 34.]
   ...
   [ 6.  6.  6.]
   [ 6.  6.  6.]
   [ 6.  6.  6.]]

  [[38. 38. 38.]
   [41. 41. 41.]
   [35. 35. 35.]
   ...
   [ 6.  6.  6.]
   [ 6.  6.  6.]
   [ 6.  6.  6.]]

  ...

  [[33. 33. 33.]
   [36. 36. 36.]
   [39. 39. 39.]
   ...
   [49. 49. 49.]
   [43. 43. 43.]
   [39. 39. 39.]]

  [[34. 34. 34.]
   [37. 37. 37.]
   [40. 40. 40.]
   ...
   [47. 47. 47.]
   [41. 41. 41.]
   [36. 36. 36.]]

  [[34. 34. 34.]
   [38. 38. 38.]
   [42. 42. 42.]
   ...
   [45. 45. 45.]
   [39. 39. 39.]
   [33. 33. 33.]]]


 [[[ 4.  4.  4.]
   [ 5.  5.  5.]
   [ 7.  7.  7.]
   ...
   [ 2.  2.  2.]
   [ 2.  2.  2.]
   [ 2.  2.  2.]]

  [[ 4.  4.  4.]
   [ 5.  5.  5.]
   [ 7.  7.  7.]
   ...
   [ 2.  2.  2.]
   [ 2.  2.  2.]
   [ 2.  2.  2.]]

  [[ 4.  4.  4.]
   [ 5.  5.  5.]
   [ 7.  7.  7.]
   ...
   [ 3.  3.  3.]
   [ 3.  3.  3.]
   [ 3.  3.

In [ ]:
#es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.01)

model.fit(X, Y, epochs=100, validation_data=(X_val, Y_val), batch_size=8)

Epoch 1/100
11/11 [==============================] - 11s 799ms/step - loss: 7699.0889 - accuracy: 0.5814 - val_loss: 2840.6968 - val_accuracy: 0.1905
Epoch 2/100
11/11 [==============================] - 7s 625ms/step - loss: 4036.6399 - accuracy: 0.3605 - val_loss: 2152.5840 - val_accuracy: 0.5714
Epoch 3/100
11/11 [==============================] - 7s 631ms/step - loss: 3043.5085 - accuracy: 0.5000 - val_loss: 1424.2131 - val_accuracy: 0.0476
Epoch 4/100
11/11 [==============================] - 7s 634ms/step - loss: 2076.1169 - accuracy: 0.3140 - val_loss: 1401.7922 - val_accuracy: 0.5238
Epoch 5/100
11/11 [==============================] - 7s 627ms/step - loss: 1352.0249 - accuracy: 0.4535 - val_loss: 1518.7061 - val_accuracy: 0.5714
Epoch 6/100
11/11 [==============================] - 7s 637ms/step - loss: 631.9720 - accuracy: 0.6047 - val_loss: 1377.5913 - val_accuracy: 0.5238
Epoch 7/100
11/11 [==============================] - 7s 628ms/step - loss: 524.0015 - accuracy: 0.6395 - v

### Testing Random Images

Try changing the sample number below

In [ ]:
X_test = []
Y_test = []
X_test, Y_test = importImage("/content/drive/MyDrive/Traing Data Preprocessing/Training Data/Grid/sample_98.jpg", X_test, Y_test)
X_test = np.array(X_test)
Y_predict = model.predict(X_test)
Y_predict = [[round(Y_predict[0][0]), round(Y_predict[0][1]), round(Y_predict[0][2]), round(Y_predict[0][3]), round(Y_predict[0][4]), round(Y_predict[0][5]), round(Y_predict[0][6]), round(Y_predict[0][7]), round(Y_predict[0][8])]]
print(Y_predict)
print(Y_test)

[[3, 38, 43, 283, 33, 44, 193, 293, 187]]
[[1, 41, 43, 284, 37, 45, 193, 291, 189]]


In [ ]:
model.save(MODEL_TF)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)

INFO:tensorflow:Assets written to: models/model/assets


2299760

In [ ]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net